# Enterprise Data Science

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
from sklearn import linear_model

from datetime import datetime

from scipy import signal


pd.set_option('display.max_rows',500)

![Crisp_DM](CRISP_DM.png)

# Data Preparation 
# Group by apply on large Data set

## Assumptions are that the data frame are with sorted Dates

In [2]:
pd_JH_data=pd.read_csv('C:/Users/jitin/ads_covid-19/data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0
1,2020-01-22,no,"Korea, South",1
2,2020-01-22,no,Kosovo,0
3,2020-01-22,no,Kuwait,0
4,2020-01-22,no,Kyrgyzstan,0


# Test Data Set

In [3]:

test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-04-01')]

In [4]:
#John Hopkins Data
test_data.head()

,date,state,country,confirmed
18929,2020-04-02,no,Germany,84794
18969,2020-04-02,no,US,244610
19195,2020-04-03,no,Germany,91159
19235,2020-04-03,no,US,276547
19460,2020-04-04,no,Germany,96092


In [5]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-09-13,no,261737
US,2020-09-13,no,6519573


In [6]:
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [7]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-09-13     261737
      US      2020-09-13    6519573

In [8]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [9]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  18929            NaN
                19195            NaN
                19460      16.052694
                19727      21.372453
                19993      27.427355
                20259      27.511936
                20525      21.792179
                20791      21.495848
                21057      26.565108
                21323      36.198404
                21588      43.982990
                21855      49.423444
                22121      74.043747
                22386      56.424411
                22654      42.468318
                22919      41.525988
                23185      49.898063
                23451      75.684007
                23716      78.000000
                23983      94.526338
                24248      82.985208
                24514      62.294061
                24781      70.294338
                25047      91.536840
                25312     112.903164
                25577     140.472457
                

In [10]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [11]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,966,NaN
4,Alberta,Canada,1263,NaN


In [12]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0
1,1,2020-01-22,no,"Korea, South",1
2,2,2020-01-22,no,Kosovo,0
3,3,2020-01-22,no,Kuwait,0
4,4,2020-01-22,no,Kyrgyzstan,0


In [13]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0,NaN
1,1,2020-01-22,no,"Korea, South",1,NaN
2,2,2020-01-22,no,Kosovo,0,NaN
3,3,2020-01-22,no,Kuwait,0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0,NaN


In [14]:
pd_result_larg[pd_result_larg['country']=='Germany']

,index,date,state,country,confirmed,confirmed_DR
43,43,2020-01-22,no,Germany,0,NaN
308,308,2020-01-23,no,Germany,0,NaN
575,575,2020-01-24,no,Germany,0,NaN
841,841,2020-01-25,no,Germany,0,NaN
1107,1107,2020-01-26,no,Germany,0,NaN
1373,1373,2020-01-27,no,Germany,1,0.666667
1638,1638,2020-01-28,no,Germany,4,0.833333
1905,1905,2020-01-29,no,Germany,4,2.000000
2170,2170,2020-01-30,no,Germany,4,inf
2437,2437,2020-01-31,no,Germany,5,8.666667


## Filtering the Data with groupby apply

In [15]:
def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [16]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [17]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1,NaN,0.8
2,2,2020-01-22,no,Kosovo,0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0,NaN,0.0


## Filtered Doubling Rate

In [18]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
62771,no,Zimbabwe,61540,57.891362
62772,no,Zimbabwe,61776,86.626761
62773,no,Zimbabwe,62072,213.982759
62774,no,Zimbabwe,62308,284.880000
62775,no,Zimbabwe,62775,301.361446


In [19]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
62771,62771,2020-09-13,no,Barbados,181,360.666667,180.6,902.000000
62772,62772,2020-09-13,no,Belarus,74173,380.346187,74170.2,384.099688
62773,62773,2020-09-13,no,Belgium,93455,96.444213,93433.4,97.997033
62774,62774,2020-09-13,no,Albania,11353,67.387550,11349.2,68.927295
62775,62775,2020-09-13,no,Zimbabwe,7526,319.333333,7528.8,301.361446


In [20]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)

In [21]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()


,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
61487,61487,2020-09-09,no,Germany,256433,181.692436,256580.0,168.922815
61755,61755,2020-09-10,no,Germany,258149,160.722431,258018.2,168.789051
62019,62019,2020-09-11,no,Germany,259735,156.332930,259374.2,184.661656
62287,62287,2020-09-12,no,Germany,260817,194.577961,260701.8,193.296120
62552,62552,2020-09-13,no,Germany,261737,260.502498,262029.4,196.370744


In [22]:
pd_result_larg.to_csv('C:/Users/jitin/ads_covid-19/data/processed/COVID_final_set.csv',sep=';',index=False)